The rating system used in the simulation competition is highly dependent on the most recent performance. When humans play the game, this dependency is necessary because we need to take into account the improvement of their skills. However, agents do not improve, so this mechanism may not be necessary for the simulation competition. So I calculated a rating without that dependency by maximum likelihood estimation. This rating is based on the following assumptions:

- The probability that an agent with a rating of x wins against an agent with a rating of y is $ 1 / (10^{(y-x)/400} + 1) $.
- The ratings follow a normal distribution and their mean is equal to the mean of the real ratings.

The first assumption is the same as the one used in the Elo rating, and is probably used in this competition as well.
The second is to prevent the ratings of all winning and all losing agents from going to infinity. I had a feeling that the unimodality of the likelihood function would not be guaranteed if we estimated the rating and standard deviation at the same time, but I tried it several times and got the same result, so it is probably okay.

**UPD**: It turned out that it was not okay, so sigma was fixed.

In [ ]:
!pip install kaggle-environments --upgrade

In [ ]:
import os
import sys
from time import time, sleep
import json
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")
from operator import itemgetter
from itertools import groupby, count
from collections import defaultdict, Counter

import numpy as np
import numba
from scipy import optimize, integrate
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from kaggle_environments import (
    evaluate, make, utils,
    get_episode_replay, list_episodes, list_episodes_for_submission  # list_episodes_for_team is no longer available
)

COMPETITION_ID = 22838
COMPETITION = "rock-paper-scissors"
pd.options.display.max_rows = 1000

In [ ]:
df_episode_agents = pd.read_csv("../input/meta-kaggle/EpisodeAgents.csv")

df_episodes = pd.read_csv("../input/meta-kaggle/Episodes.csv")
df_episodes["CreateTime"] = pd.to_datetime(df_episodes["CreateTime"], format="%m/%d/%Y %H:%M:%S")
df_episodes["EndTime"] = pd.to_datetime(df_episodes["EndTime"], format="%m/%d/%Y %H:%M:%S")

df_teams = pd.read_csv("../input/meta-kaggle/Teams.csv")
df_teams["ScoreFirstSubmittedDate"] = pd.to_datetime(df_teams["ScoreFirstSubmittedDate"], format="%m/%d/%Y")
df_teams["LastSubmissionDate"] = pd.to_datetime(df_teams["LastSubmissionDate"], format="%m/%d/%Y")
df_teams["MedalAwardDate"] = pd.to_datetime(df_teams["MedalAwardDate"], format="%m/%d/%Y")

In [ ]:
df_episodes = df_episodes[df_episodes["CompetitionId"]==COMPETITION_ID]
df_episodes.reset_index(drop=True, inplace=True)

set_episode_ids = set(df_episodes["Id"].tolist())
df_episode_agents = df_episode_agents[df_episode_agents["EpisodeId"].isin(set_episode_ids)]
df_episode_agents.reset_index(drop=True, inplace=True)

df_teams = df_teams[df_teams["CompetitionId"]==COMPETITION_ID]
df_teams.reset_index(drop=True, inplace=True)

In [ ]:
df_episode_agents = df_episode_agents[df_episode_agents["EpisodeId"].map(df_episode_agents["EpisodeId"].value_counts())==2]
df_episode_agents.sort_values(["EpisodeId", "Index"], inplace=True)
df_episode_agents.reset_index(drop=True, inplace=True)

def calc_win_lose_draw(a, b):
    if np.isnan(a):
        a = -9999999
    if np.isnan(b):
        b = -9999999
    if a == b:
        return 0
    if a > b:
        return 1
    if a < b:
        return -1
    assert False

win_lose_draw = np.empty(len(df_episode_agents), dtype=np.int64)
win_lose_draw[::2] = np.vectorize(calc_win_lose_draw)(df_episode_agents.loc[::2, "Reward"].values, df_episode_agents.loc[1::2, "Reward"].values)
win_lose_draw[1::2] = -win_lose_draw[::2]
df_episode_agents["WinLoseDraw"] = win_lose_draw

set_validated_submissions = set(df_episode_agents.dropna()["SubmissionId"].tolist())
print(f"num of valid submissions = {len(set_validated_submissions)}")

label_encoder_sub_id = LabelEncoder()
label_encoder_sub_id.fit(df_episode_agents.dropna(subset=["InitialScore"])["SubmissionId"])
# df_episode_agents["LabelEncodedSubmissionId"] = label_encoder_sub_id.transform(df_episode_agents["SubmissionId"])
# df_episode_agents["OpponentLabelEncodedSubmissionId"] = df_episode_agents["LabelEncodedSubmissionId"].values[np.arange(len(df_episode_agents)) ^ 1]

In [ ]:
%%time
# get submission data

dict_submissions = {}

episode_ids_covering_all_submissions = df_episode_agents.dropna(subset=["InitialScore"]).drop_duplicates("SubmissionId").drop_duplicates("EpisodeId")["EpisodeId"].tolist()

for idx_episode_ids in range(0, len(episode_ids_covering_all_submissions), 1000):
    ids = episode_ids_covering_all_submissions[idx_episode_ids:idx_episode_ids+1000]
    print(f"{idx_episode_ids}")
    episodes = list_episodes(ids)
    sleep(5)
    for submission in episodes["result"]["submissions"]:
        dict_submissions[submission["id"]] = submission

len(dict_submissions)

In [ ]:
df = df_episode_agents.dropna(subset=["InitialScore"])
df.reset_index(drop=True, inplace=True)
assert (df.loc[::2, "EpisodeId"].values == df.loc[1::2, "EpisodeId"].values).all()
df["LabelEncodedSubmissionId"] = label_encoder_sub_id.transform(df["SubmissionId"])
win_idxs = df.loc[(df["WinLoseDraw"]==1), "LabelEncodedSubmissionId"].values
lose_idxs = df.loc[(df["WinLoseDraw"]==-1), "LabelEncodedSubmissionId"].values
draw_idxs = df.loc[(df["WinLoseDraw"]==0), "LabelEncodedSubmissionId"].values
assert len(win_idxs) == len(lose_idxs)
assert len(draw_idxs) % 2 == 0

In [ ]:
url = f'https://www.kaggle.com/c/{COMPETITION}/leaderboard.json?includeBeforeUser=true&includeAfterUser=false'
!wget '{url}' -O leaderboard.json

with open("leaderboard.json") as f:
    jsn = json.load(f)
leaderboard_data = jsn["beforeUser"] + jsn["afterUser"]
df_leaderboard = pd.DataFrame(leaderboard_data)
gold_score = min(float(x["score"]) for x in leaderboard_data if x["medal"]=="gold")
silver_score = min(float(x["score"]) for x in leaderboard_data if x["medal"]=="silver")
bronze_score = min(float(x["score"]) for x in leaderboard_data if x["medal"]=="bronze")
gold_rank = max(float(x["rank"]) for x in leaderboard_data if x["medal"]=="gold")
silver_rank = max(float(x["rank"]) for x in leaderboard_data if x["medal"]=="silver")
bronze_rank = max(float(x["rank"]) for x in leaderboard_data if x["medal"]=="bronze")
medal_thresholds = [gold_score, silver_score, bronze_score]
medal_rank_thresholds = [gold_rank, silver_rank, bronze_rank]
medal_colors = ["#B88121", "#838280", "#8E5B3D"]
medal_colors_2 = ['#B47D1D', '#7F7E7C', '#8A5739']
background_medal_colors = ["#F9F7F3", "#FAFAFA", "#F8F7F6", "#FCFCFC"]
background_medal_colors_2 = ["#F5F3EF", "#F6F6F6", "#F4F3F2", "#F8F8F8"]
medal_thresholds

In [ ]:
df_submissions = pd.DataFrame(dict_submissions.values())
df_submissions = df_submissions[["id", "dateSubmitted", "teamId"]]
df_submissions["dateSubmitted"] = df_submissions["dateSubmitted"].map(lambda x: datetime.fromtimestamp(x["seconds"]))
df_submissions["teamName"] = df_submissions["teamId"].map(dict(df_leaderboard[["teamId", "teamName"]].values))
dict_submission_id_to_current_rating = dict(df_episode_agents.drop_duplicates(keep="last", subset="SubmissionId").dropna(subset=["UpdatedScore"])[["SubmissionId", "UpdatedScore"]].values)
df_submissions["currentRating"] = df_submissions["id"].map(dict_submission_id_to_current_rating)
df_submissions["numEpisodes"] = df_submissions["id"].map(df_episode_agents.dropna(subset=["InitialScore"])["SubmissionId"].value_counts())

In [ ]:
# optimize

mu = df_submissions["currentRating"].mean()
sigma = df_submissions["currentRating"].std()

def negative_log_likelihood(x, verbose=True):
    ratings = x[:-1]
    #sigma = x[-1]
    
    # prior distribution
    a = ((ratings - mu) ** 2.0).sum() / (2.0 * sigma * sigma) \
      + len(ratings) * 0.5 * np.log(2.0 * np.pi * sigma * sigma)
    
    # Elo rating
    win_ratings = ratings[win_idxs]
    lose_ratings = ratings[lose_idxs]
    draw_ratings = ratings[draw_idxs]
    b = np.log(10.0 ** ((lose_ratings - win_ratings) / 400.0) + 1.0).sum() \
      + 0.5 * np.log(10.0 ** ((draw_ratings[::2] - draw_ratings[1::2]) / 400.0) + 1.0).sum() \
      + 0.5 * np.log(10.0 ** ((draw_ratings[1::2] - draw_ratings[::2]) / 400.0) + 1.0).sum()
    
    res = a + b
    if verbose:
        print(f"negative_log_likelihood: {res:19.7f}")
    return res

def gradient(x):
    ratings = x[:-1]
    #sigma = x[-1]
    n = len(ratings)
    
    res = np.empty_like(x)
    res[:-1] = (ratings - mu) / (sigma * sigma)
    res[-1] = 0#- ((ratings - mu) ** 2).sum() / (sigma * sigma * sigma) + n / sigma
    
    win_ratings = ratings[win_idxs]
    lose_ratings = ratings[lose_idxs]
    draw_ratings = ratings[draw_idxs]
    l = np.log(10)
    e = - l / (400.0 * (1.0 + 10.0 ** ((win_ratings - lose_ratings) / 400.0)))
    res[:-1] += np.bincount(win_idxs, e, n)
    res[:-1] -= np.bincount(lose_idxs, e, n)
    e = - 0.5 * l / (400.0 * (1.0 + 10.0 ** ((draw_ratings[::2] - draw_ratings[1::2]) / 400.0))) \
      + 0.5 * l / (400.0 * (1.0 + 10.0 ** ((draw_ratings[1::2] - draw_ratings[::2]) / 400.0)))
    res[:-1] += np.bincount(draw_idxs[::2], e, n)
    res[:-1] -= np.bincount(draw_idxs[1::2], e, n)
    return res

# initialize
x0 = np.empty(len(label_encoder_sub_id.classes_)+1)
x0[:-1] = np.random.randn(len(label_encoder_sub_id.classes_)) * 300 + 600
x0[-1] = 300
#negative_log_likelihood(x0), gradient(x0)

In [ ]:
%%time
bounds = [(None, None) for _ in range(len(x0))]
bounds[-1] = (1, None)
x, target_value, optimization_info = optimize.fmin_l_bfgs_b(negative_log_likelihood, x0, fprime=gradient, bounds=bounds, factr=1e2)
assert optimization_info["warnflag"] == 0, "not converged"
x, target_value, optimization_info

In [ ]:
dict_estimated_agent_scores = dict(zip(label_encoder_sub_id.classes_, x))
df_submissions["estimatedRating"] = df_submissions["id"].map(dict_estimated_agent_scores)

In [ ]:
plt.figure(figsize=(10, 3), dpi=150)
hist = plt.hist(df_submissions["estimatedRating"], bins=400)
plt.grid()
plt.xlabel("Estimated rating")
plt.title("Distribution of estimated ratings")
plt.show()

In [ ]:
plt.figure(figsize=(8, 8), dpi=150)
ylim = df_submissions["estimatedRating"].min()-300, df_submissions["estimatedRating"].max()+300
plt.vlines(medal_thresholds, *ylim, medal_colors, linewidth=1.5)
plt.scatter(df_submissions["currentRating"], df_submissions["estimatedRating"], s=0.1, label="Agent")
plt.text(950, 500, "Lucky", rotation=-25, fontsize=16, ha="center", va="center", bbox={"boxstyle": "rarrow", "fc": "#ffffee"})
plt.text(200, 850, "Unlucky", rotation=-25, fontsize=16, ha="center", va="center", bbox={"boxstyle": "larrow", "fc": "#eeeeff"})

plt.grid()
plt.axes().set_aspect("equal")
plt.xlabel("Current Rating")
plt.ylabel("Estimated Rating")
plt.ylim(*ylim)
plt.xticks(np.arange(-1000, 1400, 200))
plt.title("How much does the order of the episodes affect the rating?")
plt.legend()
plt.show()

# Estimated rating vs Submission date

Please compare the stability with the [one using the real ratings](https://www.kaggle.com/nagiss/rps-leaderboard-analysis#Rating-vs-Submission-date).

In [ ]:
plt.figure(figsize=(20, 70))
cmap = plt.get_cmap("tab10")
dict_team_id_to_team_rank = defaultdict(lambda: np.nan)
dict_team_id_to_team_rank.update(dict(df_leaderboard[["teamId", "rank"]].values))

xlim = datetime(2020, 11, 3), datetime(2021, 2, 3)
max_rating = (int(df_submissions["estimatedRating"].max()) // 50 + 1) * 50
min_rating = max_rating - 350

for team_id, df in df_submissions.groupby("teamId"):
    rank = dict_team_id_to_team_rank[team_id]
    if not 1 <= rank <= 100:
        continue
    plt.subplot(20, 5, rank)
    #plt.hlines(medal_thresholds, *xlim, medal_colors, linewidth=1.2, linestyles="solid")
    plt.scatter(df["dateSubmitted"], df["estimatedRating"], s=10, c=cmap((rank-1)%10))
    plt.xlim(*xlim)
    plt.ylim(min_rating, max_rating)
    team_name = df.iloc[0]["teamName"]
    plt.title(f"{rank}  {team_name}")
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(bymonthday=(1, 16)))
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    plt.grid()
    
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.3)
plt.show()

# Top agents

The list of agents with the highest estimated rating.

In [ ]:
%%time

def calc_sigma(sub_id):
    idx_x = label_encoder_sub_id.transform([sub_id])[0]
    x2 = x.copy()
    
    win_idxs_2 = win_idxs[(win_idxs==idx_x) | (lose_idxs==idx_x)]
    lose_idxs_2 = lose_idxs[(win_idxs==idx_x) | (lose_idxs==idx_x)]
    draw_idxs_2 = draw_idxs[(draw_idxs==idx_x) | (draw_idxs==idx_x).reshape(-1, 2)[:, ::-1].reshape(-1)]
    
    def negative_log_likelihood_2(x, verbose=True):
        ratings = x[:-1]
        #sigma = x[-1]

        # prior distribution
        a = ((ratings[idx_x] - mu) ** 2.0) / (2.0 * sigma * sigma) \
          + len(ratings) * 0.5 * np.log(2.0 * np.pi * sigma * sigma)

        # Elo rating
        win_ratings = ratings[win_idxs_2]
        lose_ratings = ratings[lose_idxs_2]
        draw_ratings = ratings[draw_idxs_2]
        b = np.log(10.0 ** ((lose_ratings - win_ratings) / 400.0) + 1.0).sum() \
          + 0.5 * np.log(10.0 ** ((draw_ratings[::2] - draw_ratings[1::2]) / 400.0) + 1.0).sum() \
          + 0.5 * np.log(10.0 ** ((draw_ratings[1::2] - draw_ratings[::2]) / 400.0) + 1.0).sum()

        res = a + b
        if verbose:
            print(f"negative_log_likelihood: {res:19.7f}")
        return res
    
    offset = negative_log_likelihood_2(x, verbose=False)
    
    cache = []
    def f_all(r):
        r = r + x[idx_x]
        x2[idx_x] = r
        p = np.exp(-negative_log_likelihood_2(x2, verbose=False) + offset)
        #print(r, p)
        cache.append((r, p))
        return p
    
    def f_mean(r):
        r = r + x[idx_x]
        x2[idx_x] = r
        p = np.exp(-negative_log_likelihood_2(x2, verbose=False) + offset)
        return p * r
    
    def f_sq_mean(r):
        r = r + x[idx_x]
        x2[idx_x] = r
        p = np.exp(-negative_log_likelihood_2(x2, verbose=False) + offset)
        return p * r * r
    
    a, ea = integrate.quad(f_all, -np.inf, np.inf, epsrel=1e-4)
#     plt.scatter([r for r, _ in cache], [p for _,p in cache])
#     plt.xlim(1000, 1800)
#     plt.show()
    m, em = integrate.quad(f_mean, -np.inf, np.inf, epsrel=1e-4)
    s, es = integrate.quad(f_sq_mean, -np.inf, np.inf, epsrel=1e-4)
    #print(a, ea, m, em, s, es)
    m /= a
    s /= a
    res = np.sqrt(s - m * m)
    return res

In [ ]:
%%time
set_top_sub_ids = set(df_submissions["id"][df_submissions["estimatedRating"].nlargest(10000).index].tolist())  # It's time-consuming, so only calculate top agents
df_submissions["estimatedSigma"] = [calc_sigma(sub_id) if sub_id in set_top_sub_ids else np.nan for sub_id in df_submissions["id"].tolist()]
df_submissions["μ-2σ"] = (df_submissions["estimatedRating"] - df_submissions["estimatedSigma"] * 2).fillna(-9999)


In [ ]:
df_submissions.sort_values("estimatedRating", ascending=False).reset_index(drop=True)[["teamName", "estimatedRating", "currentRating", "dateSubmitted", "estimatedSigma", "numEpisodes", "μ-2σ", "id"]].head(100).style.format({
    "estimatedRating": lambda x: f"{x:7.2f}",
    "currentRating": lambda x: f"{x:7.2f}",
    "id": lambda x: f'<a href="https://www.kaggle.com/c/{COMPETITION}/leaderboard?dialog=episodes-submission-{x}">{x}</a>',
    "estimatedSigma": lambda x: f"{x:7.2f}",
    "μ-2σ": lambda x: f"{x:7.2f}",
    "dateSubmitted": lambda x: x.strftime("%Y-%m-%d")}
).background_gradient(subset=["estimatedRating", "currentRating", "μ-2σ"])


# Leaderboard based on max estimated rating

Make a leaderboard using estimated ratings instead of real ratings.

In [ ]:
df_rankings = df_submissions.groupby("teamId").agg(
    teamName=("teamName", "first"),
    numAgents=("teamId", "count"),
    maxCurrentRating=("currentRating", "max"),
    maxEstimatedRating=("estimatedRating", "max"),
    last5EstimatedRating=("estimatedRating", lambda x: x[-5:].mean()),
    last30EstimatedRating=("estimatedRating", lambda x: x[-30:].mean()),
    top5EstimatedRating=("estimatedRating", lambda x: np.mean(sorted(x)[-5:])),
    top30EstimatedRating=("estimatedRating", lambda x: np.mean(sorted(x)[-30:])),
    maxMuMin2Sigma=("μ-2σ", "max"),
)
# df_rankings["realRank"] = df_rankings["maxCurrentRating"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["realRank"] = df_rankings.index.map(dict(df_leaderboard[["teamId", "rank"]].values)).fillna(9999).astype(np.int64)
df_rankings["top1Rank"] = df_rankings["maxEstimatedRating"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["last5Rank"] = df_rankings["last5EstimatedRating"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["last30Rank"] = df_rankings["last30EstimatedRating"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["top5Rank"] = df_rankings["top5EstimatedRating"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["top30Rank"] = df_rankings["top30EstimatedRating"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["top1_μ-2σRank"] = df_rankings["maxMuMin2Sigma"].rank(method="min", ascending=False).astype(np.int64)
df_rankings["teamMembers"] = df_rankings.index.map(dict(df_leaderboard[["teamId", "teamMembers"]].values))
#df_rankings.sort_values("realRank")

In [ ]:
def background_color(df, key):
    def rank_to_color(rank):
        for medal in range(3):
            if rank <= medal_rank_thresholds[medal]:
                return medal_colors[medal] if rank % 2 else medal_colors_2[medal]
        return "#FDFDFD" if rank % 2 else "#F9F9F9"
    def rank_to_background_color(rank):
        for medal in range(3):
            if rank <= medal_rank_thresholds[medal]:
                return background_medal_colors[medal] if rank % 2 else background_medal_colors_2[medal]
        return background_medal_colors[3] if rank % 2 else background_medal_colors_2[3]
    def css(color, text_color_threshold=0.408):
        def relative_luminance(color):
            rgba = int(color[1:3], 16) / 255, int(color[3:5], 16) / 255, int(color[5:7], 16) / 255
            r, g, b = (
                x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055 ** 2.4)
                for x in rgba[:3]
            )
            return 0.2126 * r + 0.7152 * g + 0.0722 * b
        dark = relative_luminance(color) < text_color_threshold
        text_color = "#f1f1f1" if dark else "#000000"
        return f"background-color: {color};color: {text_color};"
    return pd.DataFrame(
        [[css(rank_to_color(rank))] + [css(rank_to_background_color(rank))]*(df.shape[1]-1) for rank in df[key].tolist()],
        index=df.index,
        columns=df.columns,
    )

df = df_submissions.sort_values("estimatedRating", ascending=False).drop_duplicates(subset="teamId", keep="first")
df.reset_index(drop=True, inplace=True)
df["estimatedRatingRank"] = np.arange(1, len(df)+1)
df["teamMembers"] = df["teamId"].map(dict(df_leaderboard[["teamId", "teamMembers"]].values))
df["realRank"] = df["teamId"].map(df_rankings["realRank"])
df[["estimatedRatingRank", "realRank", "teamName", "teamMembers", "estimatedRating", "currentRating", "estimatedSigma", "id", "dateSubmitted", "numEpisodes"]][:300] \
.rename(columns={
    "estimatedRatingRank": "top1Rank",
    "id": "submission Id",
    "currentRating": "currentRatingOf MaxEstiRatingAgent",
    "estimatedRating": "estimated Rating",
    "estimatedSigma": "estimated Sigma",
    "dateSubmitted": "date Submitted",
    "numEpisodes": "num Episodes"
}).style.format({
    "teamMembers": lambda x: "".join(f'<div style="float: right; margin: -4px 2px;"><a href="https://www.kaggle.com{xi["profileUrl"]}"><img src="{xi["thumbnailUrl"]}" width="24" height="24" alt="{xi["displayName"]}"></a></div>' for xi in x),
    "estimatedRating": lambda x: f"{x:7.2f}",
    "currentRatingOf MaxEstiRatingAgent": lambda x: f"{x:7.2f}",
    "estimated Sigma": lambda x: f"{x:7.2f}",
    "submission Id": lambda x: f'<a href="https://www.kaggle.com/c/{COMPETITION}/leaderboard?dialog=episodes-submission-{x}">{x}</a>',
    "date Submitted": lambda x: x.strftime("%Y-%m-%d")}
).set_table_styles([{"selector": "th", "props": [("max-width", "90px")]}]) \
.apply(background_color, axis=None, key="top1Rank").hide_index()

# Leaderboard based on last 5 agents

Each team will be ranked according to the average estimated ratings of the last five agents.  
Teams with high performance variance will be disadvantaged, and teams with a low number of submissions will be advantaged.

In [ ]:
df_rankings.reset_index().sort_values("last5Rank")[["last5Rank", "realRank", "teamName", "teamMembers", "last5EstimatedRating", "numAgents"]][:300].style.format({
    "teamMembers": lambda x: "".join(f'<div style="float: right; margin: -4px 2px;"><a href="https://www.kaggle.com{xi["profileUrl"]}"><img src="{xi["thumbnailUrl"]}" width="24" height="24" alt="{xi["displayName"]}"></a></div>' for xi in x),
    "last5EstimatedRating": lambda x: f"{x:7.2f}",
}).set_table_styles([{"selector": "th", "props": [("max-width", "150px")]}]) \
.apply(background_color, axis=None, key="last5Rank").hide_index()

# Leaderboard based on top 5 agents

Rank each team by the average estimated rating of their best five agents.

In [ ]:
df_rankings.reset_index().sort_values("top5Rank")[["top5Rank", "realRank", "teamName", "teamMembers", "top5EstimatedRating", "numAgents"]][:300].style.format({
    "teamMembers": lambda x: "".join(f'<div style="float: right; margin: -4px 2px;"><a href="https://www.kaggle.com{xi["profileUrl"]}"><img src="{xi["thumbnailUrl"]}" width="24" height="24" alt="{xi["displayName"]}"></a></div>' for xi in x),
    "top5EstimatedRating": lambda x: f"{x:7.2f}",
}).set_table_styles([{"selector": "th", "props": [("max-width", "150px")]}]) \
.apply(background_color, axis=None, key="top5Rank").hide_index()

# Comparison

In [ ]:
def background_color_comparison(df):
    def rank_to_color(rank, idx):
        for medal in range(3):
            if rank <= medal_rank_thresholds[medal]:
                return medal_colors[medal] if idx % 2 else medal_colors_2[medal]
        return "#FDFDFD" if idx % 2 else "#F9F9F9"
    def rank_to_background_color(rank):
        for medal in range(3):
            if rank <= medal_rank_thresholds[medal]:
                return background_medal_colors[medal] if rank % 2 else background_medal_colors_2[medal]
        return background_medal_colors[3] if rank % 2 else background_medal_colors_2[3]
    def css(color, text_color_threshold=0.408):
        def relative_luminance(color):
            rgba = int(color[1:3], 16) / 255, int(color[3:5], 16) / 255, int(color[5:7], 16) / 255
            r, g, b = (
                x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055 ** 2.4)
                for x in rgba[:3]
            )
            return 0.2126 * r + 0.7152 * g + 0.0722 * b
        dark = relative_luminance(color) < text_color_threshold
        text_color = "#f1f1f1" if dark else "#000000"
        return f"background-color: {color};color: {text_color};"
    res = pd.DataFrame(
        [[css(rank_to_background_color(rank))]*(df.shape[1]) for rank in df["real Rank"].tolist()],
        index=df.index,
        columns=df.columns,
    )
    for col in df.columns:
        if col.endswith("Rank"):
            res[col] = [css(rank_to_color(r, i)) for i, r in enumerate(df[col].values, 1)]
    return res

df_rankings.sort_values("realRank").reset_index()[["realRank", "teamName", "teamMembers", "last5Rank", "last30Rank", "top1Rank", "top5Rank", "top30Rank", "top1_μ-2σRank", "numAgents"]][:300] \
.rename(columns={
    "realRank": "real Rank",
    "last5Rank": "last5 Rank",
    "last30Rank": "last30 Rank",
    "top1Rank": "top1 Rank",
    "top5Rank": "top5 Rank",
    "top30Rank": "top30 Rank",
    "numAgents": "num Agents",
    "top1_μ-2σRank": "top1 μ-2σ Rank",
}).style.format({
    "teamMembers": lambda x: "".join(f'<div style="float: right; margin: -4px 2px;"><a href="https://www.kaggle.com{xi["profileUrl"]}"><img src="{xi["thumbnailUrl"]}" width="24" height="24" alt="{xi["displayName"]}"></a></div>' for xi in x),
}).set_table_styles([{"selector": "th", "props": [("max-width", "50px")]}]) \
.apply(background_color_comparison, axis=None).hide_index()

In [ ]:
df_submissions.to_csv("submissions.csv", index=False)
df_rankings.to_csv("rankings.csv", index=False)